In [830]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd
import os
from matplotlib import pyplot as plt

In [831]:
DATA_DIR = '.\data\enhanced'
hairdryer = 'hair_dryer_finished.csv'
microwave = 'microwave_finished.csv'
pacifier = 'pacifier_finished.csv'

In [859]:
pdata = pd.read_csv(os.path.join(DATA_DIR, pacifier))
hdata = pd.read_csv(os.path.join(DATA_DIR, hairdryer))
mdata = pd.read_csv(os.path.join(DATA_DIR, microwave))

pdata = pdata.dropna(axis=0,how='any').drop(pdata.tail(1).index)
pdata.review_date = pd.to_datetime(pdata.review_date, format='%m/%d/%Y')
hdata = hdata.dropna(axis=0,how='any')
hdata.review_date = pd.to_datetime(hdata.review_date, format='%m/%d/%Y')
mdata = mdata.dropna(axis=0,how='any')
mdata.review_date = pd.to_datetime(mdata.review_date, format='%m/%d/%Y')

In [860]:
pdata.sort_values('review_date', inplace=True)
hdata.sort_values('review_date', inplace=True)
mdata.sort_values('review_date', inplace=True)

In [867]:
def lamk(t, param, data):
    
    times = data[:,0]
    types = data[:,1]
    
    n = data.shape[0]
    K = np.max(types)+1 #类别从0开始以数字进行标注
    s = param[0]
    #print(s)
    mu = param[1]
    delta = param[2:2+K]
    alpha = param[2+K:]
    
    temp = mu
    idx = times < t

    times = times[idx]
    types = types[idx]
    
    for i in range(times.shape[0]):
        #print(delta.shape)
        temp += alpha[types[i]] * np.exp(-delta[types[i]] * (t-times[i]))
    
    #print(s*np.log(1+np.exp(temp/s)))
    return s*np.log(1+np.exp(temp/s))

#def slam(t, param, data):


def MC(param, data):
    times = data[:,0]
    types = data[:,1]
    T = np.max(times)
    K = np.max(types) #这里的K和上面的K不一样

    r1 = 0
    for i in range(1000):
        t = np.random.uniform(0,T)
        for j in range(K):
            r1 += lamk(t, param[types[j],:], data)

    return T*r1/times.shape[0]
            
def log_likeli(param, data):
    times = data[:,0]
    types = data[:,1]
    k = np.max(types)+1
    param = param.reshape((2*k+2, k))
    param = param.T

    ll = 0
    #print(times.shape[0])
    for i in range(times.shape[0]):
        ll += np.log(lamk(times[i], param[types[i],:], data))
        
    ll -= MC(param ,data)
    print(ll)
    return ll

In [868]:
def extract_param(param):
    s = param[:, 0]
    mu = param[:, 1]
    delta = param[:, 2:2+K-1]
    alpha = param[:, 2+K:]
    return alpha, delta, mu, s

def combine(s, mu, delta, alpha):
    return np.row_stack((s,mu,delta,alpha))

In [869]:
def HP(dataset, name, param):
    product = dataset.product_title.value_counts().index[0]
    subset = dataset[dataset.product_title==product]
    subset['relative'] = [(x - subset.iloc[0]['review_date']).days for x in subset['review_date']]
    data = subset[['relative', 'like']]
    data = data[data['like']!=0]
    data = np.array(data)[:, :]
    data[:,1] = (data[:,1]+1)/2
    K = np.max(data[:,1])+1

    param = np.reshape(param, (1,K*K*2+K*2))


    best = 100000
    for i in range(1):
        param = minimize(lambda x: -log_likeli(x, data), 
                         x0=param, method='Nelder-Mead', #BFGS Nelder-Mead CG
                         options={'disp': False, 'maxiter':100})
        if param.fun < best:
            best = param.fun
            bp = param
        
    return bp

In [870]:
s = np.random.uniform(0,1,(1,2))
mu = np.random.uniform(-1,(1,2))
delta = np.random.uniform(-1,1,(2,2))
alpha = np.random.uniform(-1,1,(2,2))
param = combine(s,mu,delta,alpha)

In [871]:
param

array([[ 0.48468701,  0.18274212],
       [ 0.03056859, -0.40719546],
       [ 0.68552093,  0.78530748],
       [-0.34354615, -0.46798476],
       [-0.68032933,  0.9496365 ],
       [ 0.69518695, -0.05353631]])

In [872]:
#pparam = HP(hdata, 'hair', param)
hparam = HP(hdata, 'hair', param)
#mparam = HP(hdata, param, 'hair')

D:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in log
D:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in double_scalars


nan
nan


KeyboardInterrupt: 

In [ ]:
pparam.x

In [ ]:
a = np.zeros(5)
a

In [694]:
a.resize((1,5))

In [702]:
a.resize((5,1))

In [703]:
a

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])